In [24]:
import csv
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [25]:
from feature import Feature
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatool.preprocess import  Preprocessor
from datatool.maneger import DataManager

In [26]:
path = "../corpus/data_augment.csv"

In [40]:
import spacy

In [27]:
data = []
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [28]:
def make_classes_dict(data):
    classes_dict = dict()
    for row in data:
        c = row[1]
        if c not in classes_dict:
            classes_dict[c] = len(classes_dict)
    return classes_dict

In [29]:
def class_filter(data, remain_classes:list):
    new_data = []
    for row in data:
        c = row[1]
        if c in remain_classes:
            new_data.append(row)
    return new_data

In [30]:
classes_dict_ = make_classes_dict(data)

In [31]:
remain_classes = "how what when where who why YN plain".split()
data_n = class_filter(data, remain_classes)
classes_dict = dict(zip(remain_classes, list(range(len(remain_classes)))))

In [32]:
def extract_X_y(data, classes_dict):
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(classes_dict[d[1]])
    return X, y

In [33]:
import random
def extract_X_y_limit(data, classes_dict:dict, limit=400):
    data_ = random.sample(data, len(data))
    X = []
    y = []
    each_len = dict(zip(classes_dict.keys(), [0]*len(classes_dict)))
    for d in data_:
        if each_len[d[1]] <= limit:
            X.append(d[0])
            y.append(classes_dict[d[1]])
            each_len[d[1]] += 1
    return X, y

In [34]:
# X, y = extract_X_y(data_n, classes_dict)

In [35]:
X, y = extract_X_y_limit(data_n, classes_dict)

In [36]:
len(X)

3208

In [37]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [39]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

OSError: [E049] Can't find spaCy data directory: 'None'. Check your installation and permissions, or use spacy.util.set_data_path to customise the location if necessary.

In [ ]:
F_path = "../X_y_data/"
F_name = "typeClassify_F2.pickle"
# import dill
import pickle
# featureM = DataManager(F_path, format_="dill")
with open(F_path+F_name, "wb") as f:
    pickle.dump(F, f)

In [ ]:
lr = LogisticRegression(solver='sag', max_iter=100)
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
print(classes_dict)
dis = 30
for y_p, x_s in zip(y_pred[:dis], X_test_str[:dis]):
    print("{0} : {1}".format(y_p, x_s))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

In [ ]:
model_path = "../models/"
model_name = "typeClassify_M2.pickle"
modelM = DataManager(model_path)
print(model_name)

In [ ]:
modelM.save_data(model_name, lr)